In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('restaurants.csv')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                360 non-null    object 
 1   address             360 non-null    object 
 2   longitude           360 non-null    float64
 3   latitude            360 non-null    float64
 4   rating              360 non-null    float64
 5   price_level         330 non-null    float64
 6   user_ratings_total  360 non-null    int64  
 7   open_now            360 non-null    bool   
 8   types               360 non-null    object 
 9   category            360 non-null    object 
dtypes: bool(1), float64(4), int64(1), object(4)
memory usage: 25.8+ KB


In [38]:
#print all these null columns
print(df.isnull().sum())

name                   0
address                0
longitude              0
latitude               0
rating                 0
price_level           30
user_ratings_total     0
open_now               0
types                  0
category               0
dtype: int64


In [39]:
#delete rows with the same address
df_filter = df.drop_duplicates(subset='address')
print(df_filter)
df_filter.to_csv('restaurants_clean.csv', index=False)

                          name                          address   longitude  \
0        Golden Dragon Chinese       1754 S Grand Ave, Glendora -117.872205   
1               Century Dragon      3711 Magnolia Blvd, Burbank -118.347561   
2    China garden Chinese food      1211 S Soto St, Los Angeles -118.219812   
3                   Fat Dragon    3500 Sunset Blvd, Los Angeles -118.276850   
4                   Sea Dragon     101 Vermont Ave, Los Angeles -118.292248   
..                         ...                              ...         ...   
355    WeHo Thai Noodle & Rice    7075 Sunset Blvd, Los Angeles -118.343966   
356         SPICY NOODLE HOUSE    68 Rio Rancho Rd #106, Pomona -117.760747   
357            Qin West Noodle  1767 Westwood Blvd, Los Angeles -118.438224   
358       Wen Hui Noodle Hours  644 W Garvey Ave, Monterey Park -118.131486   
359               Noodle House  958 E Garvey Ave, Monterey Park -118.109625   

      latitude  rating  price_level  user_ratings_t

In [40]:
df_clean = pd.read_csv('restaurants_clean.csv')
print(df_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                337 non-null    object 
 1   address             337 non-null    object 
 2   longitude           337 non-null    float64
 3   latitude            337 non-null    float64
 4   rating              337 non-null    float64
 5   price_level         310 non-null    float64
 6   user_ratings_total  337 non-null    int64  
 7   open_now            337 non-null    bool   
 8   types               337 non-null    object 
 9   category            337 non-null    object 
dtypes: bool(1), float64(4), int64(1), object(4)
memory usage: 24.2+ KB
None


In [41]:
df_clean['price_level'] = df_clean['price_level'].fillna(0)

In [42]:
df_clean.price_level.value_counts()

price_level
1.0    171
2.0    138
0.0     27
3.0      1
Name: count, dtype: int64

In [43]:
# print out how many rows in dataset when category == drinking
num_rows = df[df['category'] == 'drinking'].shape[0]
print(num_rows)

48


In [56]:
# create a location dataframe with all address in df_clean
df_clean[['street', 'county']] = df_clean['address'].str.split(',', n=1, expand=True)
df_clean['county'] = df_clean['county'].str.strip()
df_clean['street'] = df_clean['street'].str.strip()

print(df_clean)

# create a new dataframe with only street, city, longitude, latitude
location_df = df_clean[['street', 'county', 'longitude', 'latitude']].copy()
location_df['longitude'] = df_clean['longitude']
location_df['latitude'] = df_clean['latitude']


location_df.to_csv('location.csv', index=False)


                          name                          address   longitude  \
0        Golden Dragon Chinese       1754 S Grand Ave, Glendora -117.872205   
1               Century Dragon      3711 Magnolia Blvd, Burbank -118.347561   
2    China garden Chinese food      1211 S Soto St, Los Angeles -118.219812   
3                   Fat Dragon    3500 Sunset Blvd, Los Angeles -118.276850   
4                   Sea Dragon     101 Vermont Ave, Los Angeles -118.292248   
..                         ...                              ...         ...   
332    WeHo Thai Noodle & Rice    7075 Sunset Blvd, Los Angeles -118.343966   
333         SPICY NOODLE HOUSE    68 Rio Rancho Rd #106, Pomona -117.760747   
334            Qin West Noodle  1767 Westwood Blvd, Los Angeles -118.438224   
335       Wen Hui Noodle Hours  644 W Garvey Ave, Monterey Park -118.131486   
336               Noodle House  958 E Garvey Ave, Monterey Park -118.109625   

      latitude  rating  price_level  user_ratings_t

In [57]:
reviews_df = df[['name', 'rating', 'user_ratings_total']].copy()
reviews_df = reviews_df.rename(columns={'user_ratings_total': 'total_reviews'})
reviews_df.to_csv('reviews.csv', index=False)

In [58]:
restaurants_df = df[['name', 'address', 'rating', 'price_level', 'open_now']].copy()
restaurants_df.to_csv('restaurants.csv', index=False)

In [63]:
!pip3 install sqlalchemy
!pip3 install psycopg2-binary

  Using cached psycopg2_binary-2.9.9-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.4 kB)
Using cached psycopg2_binary-2.9.9-cp312-cp312-macosx_11_0_arm64.whl (2.6 MB)


In [64]:
from sqlalchemy import create_engine # type: ignore
engine = create_engine('postgresql://postgres@localhost:5432/food') # type: ignore
con = engine.connect()
location_df.to_sql('location', con, if_exists='append',index=False,chunksize=10000)
reviews_df.to_sql('review', con, if_exists='append',index=False,chunksize=10000)
restaurants_df.to_sql('restaurant', con, if_exists='append',index=False,chunksize=10000)


con.close()